# Building CLIP model using TT-NN

CLIP <GIVE REFERENCE>, is a foundational model of modern AI. 

<PUT WHAT DOES CLIP DO HERE> 

![CLIP Diagram](https://media.githubusercontent.com/media/tenstorrent/tutorial-assets/nmaurice/clip-tutorial/media/clip_tutorial/CLIP.png)

We start by importing necessary libraries

In [ ]:
import ttnn
import torch
from loguru import logger
import re
import os
import math
import numpy as np
from PIL import Image
from transformers import CLIPTokenizer, CLIPModel
import requests
from io import BytesIO

from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, InterpolationMode

We define the following helper functions.

In [ ]:

def open_ttnn():
    global device
    device = ttnn.open_device(device_id=0, l1_small_size=8192)

def close_ttnn():
    global device
    if device is not None:
        ttnn.close_device(device)

def get_device():
    global device
    return device

def convert_from_ttnn(x):
    global device
    if isinstance(x, ttnn._ttnn.tensor.Tensor):
        return ttnn.to_torch(x)
    return x

def to_ttnn(torch_tensor, dtype=None, layout=ttnn.TILE_LAYOUT):
    global device
    ttnn_tensor = ttnn.from_torch(torch_tensor, device=device, layout=layout, dtype=dtype)
    return ttnn_tensor

def to_torch_shape(ttnn_shape):
    return tuple(ttnn_shape)

# Change dtype of ttnn tensor and optionally reshape
def convert_ttnn_dtype(ttnn_tensor, dtype, new_shape=None):
    # HACK: Can't convert dtype on device
    device = get_device()
    host_tensor = ttnn.from_device(ttnn_tensor)
    host_tensor = ttnn.to_dtype(host_tensor, dtype=dtype)
    if new_shape is not None:
        host_tensor = ttnn.reshape(host_tensor, new_shape)

    return ttnn.to_device(host_tensor, device=device)


As TT-NN does not support weight loading by itself, we rely on torch.load().
We introduce the following helper function to convert weights from torch tensors to TT-NN tensors.  

In [ ]:
def convert_model_to_ttnn(state_dict):
    ttnn_state_dict = {}
    logger.info(f"Converting model to ttnn")

    # Convert state dict to ttnn tensor
    for key, value in state_dict.items():
        if isinstance(value, torch.Tensor):
            state_dict[key] = to_ttnn(value)
        elif isinstance(value, torch.Size):
            state_dict[key] = ttnn.Size(value)

    return state_dict

CLIP uses 2 types of transformers: a text transformer, and vision transformer.
To re-use code, we define a generic transformer class that will be used for both images and text. 

The transformer models used by CLIP have several layers, or resblocks, which contain the following sub-operations. 
- LayerNorm
- MultiHeadAttention (masked for text inference, unmasked for image inference)
- LayerNorm
- MLP (Linear + GELU + Linear)

The output of each block is re-used as the input of the next one. 

In [ ]:

class Transformer:
    def __init__(self, state_dict, heads, attention_mask=None, prefix=""):
        self.layers = []
        self.heads = heads
        self.attention_mask = attention_mask
        self.prefix = prefix

        layer_pattern = re.compile(f"{prefix}\.layers\.(\d+)\.")

        # Count number of layers
        layers_ids = set()
        for k in state_dict.keys():
            re_match = re.search(layer_pattern, k)
            if re_match:
                layers_ids.add(re_match.group(1))

        num_layers = len(layers_ids)

        for i in range(0, num_layers):
            resblock_prefix = f"{prefix}.layers.{i}"

            self.layers.append(
                {
                    "ln_1_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.layer_norm1.weight"], ttnn.bfloat16
                    ),
                    "ln_1_bias": convert_ttnn_dtype(state_dict[f"{resblock_prefix}.layer_norm1.bias"], ttnn.bfloat16),
                    "q_proj_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.q_proj.weight"], ttnn.bfloat16
                    ),
                    "q_proj_bias": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.q_proj.bias"], ttnn.bfloat16
                    ),
                    "k_proj_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.k_proj.weight"], ttnn.bfloat16
                    ),
                    "k_proj_bias": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.k_proj.bias"], ttnn.bfloat16
                    ),
                    "v_proj_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.v_proj.weight"], ttnn.bfloat16
                    ),
                    "v_proj_bias": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.v_proj.bias"], ttnn.bfloat16
                    ),
                    "out_proj_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.out_proj.weight"], ttnn.bfloat16
                    ),
                    "out_proj_bias": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.self_attn.out_proj.bias"], ttnn.bfloat16
                    ),
                    "ln_2_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.layer_norm2.weight"], ttnn.bfloat16
                    ),
                    "ln_2_bias": convert_ttnn_dtype(state_dict[f"{resblock_prefix}.layer_norm2.bias"], ttnn.bfloat16),
                    "mlp_c_fc_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.mlp.fc1.weight"], ttnn.bfloat16
                    ),
                    "mlp_c_fc_bias": convert_ttnn_dtype(state_dict[f"{resblock_prefix}.mlp.fc1.bias"], ttnn.bfloat16),
                    "mlp_c_proj_weight": convert_ttnn_dtype(
                        state_dict[f"{resblock_prefix}.mlp.fc2.weight"], ttnn.bfloat16
                    ),
                    "mlp_c_proj_bias": convert_ttnn_dtype(state_dict[f"{resblock_prefix}.mlp.fc2.bias"], ttnn.bfloat16),
                }
            )


    def forward(self, x):
        def mlp(x, layer):
            x = ttnn.linear(x, layer["mlp_c_fc_weight"], bias=layer["mlp_c_fc_bias"], transpose_b=True)
            x = ttnn.gelu(x)
            x = ttnn.linear(x, layer["mlp_c_proj_weight"], bias=layer["mlp_c_proj_bias"], transpose_b=True)
            return x

        def multi_head_attention(
            hidden_states,
            fused_qkv_weight,
            fused_qkv_bias,
            self_output_weight,
            self_output_bias,
            attention_mask=None,
            prefix="",
        ):
            seq_length, batch_size, hidden_size = hidden_states.shape

            self._embed_dim = hidden_size
            self._head_dim = hidden_size // self.heads
            self._scale = self._head_dim**-0.5
            self._attention_dropout = 0.0  # Unused

            compute_kernel_config = ttnn.WormholeComputeKernelConfig(
                math_fidelity=ttnn.MathFidelity.HiFi4,
                math_approx_mode=False,
                fp32_dest_acc_en=True,
                packer_l1_acc=True,
            )

            # TODO: No KV-caching for now
            (q_weights, k_weights, v_weights) = fused_qkv_weight
            (q_bias, k_bias, v_bias) = fused_qkv_bias

            # Compute Q, K, V projections
            q = ttnn.linear(hidden_states, q_weights, bias=q_bias, transpose_b=True)
            k = ttnn.linear(hidden_states, k_weights, bias=k_bias, transpose_b=True)
            v = ttnn.linear(hidden_states, v_weights, bias=v_bias, transpose_b=True)

            # Reshape to [batch_size, seq_length, num_heads, head_dim]
            q = ttnn.reshape(q, (seq_length, batch_size * self.heads, self._head_dim))
            k = ttnn.reshape(k, (seq_length, batch_size * self.heads, self._head_dim))
            v = ttnn.reshape(v, (seq_length, batch_size * self.heads, self._head_dim))

            # Transpose to [batch_size, num_heads, seq_length, head_dim] for attention computation
            q = ttnn.transpose(q, 0, 1)
            k = ttnn.transpose(k, 0, 1)
            v = ttnn.transpose(v, 0, 1)

            # Compute attention scores with proper scaling
            scores = ttnn.matmul(q, ttnn.transpose(k, -2, -1))
            scores = scores * self._scale

            # Apply attention mask if provided (matching PyTorch MHA behavior)
            if attention_mask is not None:
                # Convert attention mask to the right shape and add to scores
                # PyTorch MHA expects mask to be broadcastable to [batch_size, num_heads, seq_len, seq_len]
                scores = scores + attention_mask

            attn_weights = ttnn.softmax(
                scores, dim=-1, numeric_stable=True, compute_kernel_config=compute_kernel_config
            )

            # Apply dropout if needed (currently disabled)
            # attn_weights = ttnn.experimental.dropout(attn_weights, self._attention_dropout)

            # Apply attention weights to values
            attn_output = ttnn.matmul(attn_weights, v)

            # Reshape to [batch_size, seq_length, embed_dim]
            attn_output = ttnn.transpose(attn_output, 0, 1)
            attn_output = ttnn.reshape(attn_output, (seq_length, batch_size, self._embed_dim))

            # Apply output projection
            dense_out = ttnn.linear(
                attn_output,
                self_output_weight,
                bias=self_output_bias,
                compute_kernel_config=compute_kernel_config,
                transpose_b=True,
            )

            return dense_out

        def residual_attention_block(x, layer, i=0):
            # LayerNorm
            residual = x
            x = ttnn.layer_norm(x, weight=layer["ln_1_weight"], bias=layer["ln_1_bias"])

            # Multihead attention / Self-Attention
            # This must be equal to nn.MultiheadAttention(d_model, n_head)(x, x, x, need_weights=False, attn_mask=self.attn_mask)
            x_attn = multi_head_attention(
                x,
                fused_qkv_weight=(layer["q_proj_weight"], layer["k_proj_weight"], layer["v_proj_weight"]),
                fused_qkv_bias=(layer["q_proj_bias"], layer["k_proj_bias"], layer["v_proj_bias"]),
                self_output_weight=layer["out_proj_weight"],
                self_output_bias=layer["out_proj_bias"],
                attention_mask=self.attention_mask,
                prefix=f"{self.prefix}.layers.{i}.attn",
            )  # Vision transformer doesn't use attention mask

            x = residual + x_attn

            # LayerNorm
            x_post_ln_2 = ttnn.layer_norm(x, weight=layer["ln_2_weight"], bias=layer["ln_2_bias"])

            # Multi-Layer Perceptron
            x = x + mlp(x_post_ln_2, layer)

            return x

        for i in range(len(self.layers)):
            layer = self.layers[i]
            x = residual_attention_block(x, layer, i)

        return x

We define the VisionTransformer.

Its `forward()` method pre-process image embeddings and calls the generic transformer.

In [ ]:
class VisionTransformer:
    def __init__(self, state_dict):
        torch.manual_seed(0)
        self.output_dim = 0

        conv2_state_dict_name = "vision_model.embeddings.patch_embedding.weight"
        self.vision_width = state_dict[conv2_state_dict_name].shape[0]
        self.patch_size = state_dict[conv2_state_dict_name].shape[-1]
        self.vision_heads = self.vision_width // 64

        self.class_embedding = convert_ttnn_dtype(
            state_dict["vision_model.embeddings.class_embedding"], dtype=ttnn.bfloat16
        )
        self.positional_embedding = convert_ttnn_dtype(
            state_dict["vision_model.embeddings.position_embedding.weight"], dtype=ttnn.bfloat16
        )

        self.proj = convert_ttnn_dtype(state_dict["visual_projection.weight"], dtype=ttnn.bfloat16)

        # Weights for convolution layer
        # For sharding; use all cores; strategy = block sharding
        core_grid = ttnn.CoreGrid(x=8, y=8)
        # Error: Physical shard shape (8216, 4) must be tile {32, 32} sized
        # memory_config = ttnn.create_sharded_memory_config(conv1_weights_shape, core_grid, ttnn.ShardStrategy.HEIGHT)
        memory_config = ttnn.DRAM_MEMORY_CONFIG
        self.conv1_weights = ttnn.to_layout(
            state_dict[conv2_state_dict_name],
            layout=ttnn.ROW_MAJOR_LAYOUT,
            memory_config=memory_config,
            dtype=ttnn.bfloat16,
        )
        self.conv1_weights = convert_ttnn_dtype(self.conv1_weights, dtype=ttnn.bfloat16)

        assert self.conv1_weights.dtype == ttnn.bfloat16

        self.ln_pre_weights = state_dict["vision_model.pre_layrnorm.weight"]  # TODO: What's this ?
        self.ln_pre_bias = state_dict["vision_model.pre_layrnorm.bias"]  # TODO: What's this ?

        self.ln_post_weights = state_dict["vision_model.post_layernorm.weight"]  # TODO: What's this ?
        self.ln_post_bias = state_dict["vision_model.post_layernorm.bias"]  # TODO: What's this ?

        self.transformer = Transformer(
            state_dict, self.vision_heads, attention_mask=None, prefix="vision_model.encoder"
        )

    def forward(self, x):
        (batch_size, in_channels, height, width) = x.shape

        # Note: ttnn.conv2d uses 'Array of Struct' shape for input tensor:
        # (N, H, W, C_in)
        # whereas torch.nn.Conv2d uses 'Struct of Array' shape for input tensor:
        # (N, C_in, H, W)
        #
        # # Moreover, ttnn.conv2d produces a flattened output tensor:
        # (N, C_in, H, W) -> (1, 1, N * H * W, C_out)
        # whereas torch.nn.Conv2d produces a 4D tensor:
        # (N, C_out, H_out, W_out)

        # Also:
        # ttnn.conv2d only take a tuple for kernel_size and stride

        # Change tensor layout to (N, H, W, C_in)
        x = ttnn.permute(x, [0, 2, 3, 1])  # (N, C_in, H, W) -> (N, H, W, C_in)

        # Note: ttnn.conv2d requires row-major layout for weight tensor
        x = ttnn.to_layout(x, layout=ttnn.ROW_MAJOR_LAYOUT)

        old_memory_config = x.memory_config()
        out_channels = 768
        in_channels = 3

        bias_tensor = ttnn.zeros(
            (1, 1, 1, out_channels), dtype=ttnn.bfloat16, device=device, layout=ttnn.ROW_MAJOR_LAYOUT
        )

        x = ttnn.conv2d(
            input_tensor=x, 
            weight_tensor=self.conv1_weights,
            bias_tensor=bias_tensor,
            in_channels=in_channels,
            out_channels=out_channels,
            batch_size=batch_size,
            input_height=height,
            input_width=width,
            kernel_size=(self.patch_size, self.patch_size),
            stride=(self.patch_size, self.patch_size),
            padding=(0, 0),
            dilation=(1, 1),
            groups=0,  # No grouped convolution (?)
            device=get_device(),
            return_weights_and_bias=False,
            return_output_dim=False,
        )

        # ERROR: Number of shards along height 7 must not exceed number of cores 2
        output_height = height // self.patch_size
        output_width = width // self.patch_size

        # Check Convolution result
        x = ttnn.to_layout(x, layout=ttnn.TILE_LAYOUT)
        host_tensor = ttnn.to_torch(x, dtype=torch.float32)

        host_tensor = torch.reshape(host_tensor, (batch_size, output_height, output_width, out_channels))

        x = ttnn.reshape(x, (x.shape[0], x.shape[1] * x.shape[2], x.shape[3]))

        class_embedding = convert_ttnn_dtype(self.class_embedding, x.dtype, (x.shape[0], 1, x.shape[-1]))

        # TODO: See why we use zero tensor here and addition here ?
        zero_tensor = ttnn.zeros(
            shape=(x.shape[0], 1, x.shape[-1]), dtype=x.dtype, device=device, layout=ttnn.TILE_LAYOUT
        )

        class_embedding = ttnn.reshape(class_embedding, zero_tensor.shape)
        class_embedding = class_embedding + zero_tensor

        # TODO: Do this in L1 Sharded Memory
        # For now, move data to DRAM
        x = ttnn.to_memory_config(x, memory_config=ttnn.DRAM_MEMORY_CONFIG)

        class_embedding = ttnn.reshape(
            class_embedding, (class_embedding.shape[0], class_embedding.shape[1], class_embedding.shape[2])
        )
        class_embedding = ttnn.to_memory_config(class_embedding, memory_config=x.memory_config())

        # ERROR: RuntimeError: bad optional access (???)
        # TODO: Avoid move to host and keep on device
        x = ttnn.concat([class_embedding, x], dim=1, memory_config=None)  # shape = [*, grid ** 2 + 1, width]

        positional_embedding = convert_ttnn_dtype(self.positional_embedding, x.dtype, (1, x.shape[1], x.shape[2]))
        x = x + positional_embedding

        # LayerNorm
        x = ttnn.layer_norm(x, weight=self.ln_pre_weights, bias=self.ln_pre_bias)

        # Permute
        x = ttnn.permute(x, (1, 0, 2))  # NLD -> LND

        # Transformer
        x = self.transformer.forward(x)

        # Permute
        x = ttnn.permute(x, (1, 0, 2))  # LND -> NLD

        # LayerNorm
        x = ttnn.layer_norm(x[:, 0, :], weight=self.ln_post_weights, bias=self.ln_post_bias)

        if self.proj is not None:
            self.proj = ttnn.transpose(self.proj, 0, 1)
            x = ttnn.matmul(x, self.proj)

        return x

We then define a class for the CLIP model. 

The CLIP class instantiates:
- a Text Transformer (Transformer)
- a VisionTransformer

In [ ]:
class CLIP:
    def __init__(self, state_dict):
        self.token_embedding = convert_ttnn_dtype(
            state_dict["text_model.embeddings.token_embedding.weight"], dtype=ttnn.bfloat16
        )
        self.positional_embedding = convert_ttnn_dtype(
            state_dict["text_model.embeddings.position_embedding.weight"], dtype=ttnn.bfloat16
        )

        self.text_projection = convert_ttnn_dtype(state_dict["text_projection.weight"], dtype=ttnn.bfloat16)
        self.context_length = self.positional_embedding.shape[0]
        self.vocab_size = self.token_embedding.shape[0]
        self.transformer_width = state_dict["text_model.final_layer_norm.weight"].shape[0]
        transformer_heads = self.transformer_width // 64

        self.ln_final_weights = state_dict["text_model.final_layer_norm.weight"]
        self.ln_final_bias = state_dict["text_model.final_layer_norm.bias"]

        self.logit_scale = state_dict["logit_scale"].item()

        self.visual = VisionTransformer(state_dict)

        self.transformer = Transformer(
            state_dict, transformer_heads, attention_mask=self.build_attention_mask(), prefix="text_model.encoder"
        )
        
    def build_attention_mask(self):
        # lazily create causal attention mask, with full attention between the vision tokens
        # pytorch uses additive attention mask; fill with -inf
        mask = ttnn.full(shape=[self.context_length, self.context_length], fill_value=float("-inf"), dtype=ttnn.bfloat16, device=get_device(), layout=ttnn.TILE_LAYOUT)
        mask = ttnn.triu(mask, diagonal=1)
        return mask


    def encode_image(self, image):
        return self.visual.forward(image)

    def encode_text(self, tokens):
        tokens = convert_ttnn_dtype(tokens, dtype=ttnn.uint32)

        x = ttnn.embedding(tokens, weight=self.token_embedding, dtype=ttnn.bfloat16)

        assert x.dtype == ttnn.bfloat16

        # Add positional embedding
        x = x + self.positional_embedding

        # Permute
        x = ttnn.permute(x, (1, 0, 2))  # NLD -> LND
        x = self.transformer.forward(x)

        # Permute back
        x = ttnn.permute(x, (1, 0, 2))  # LND -> NLD

        # LayerNorm
        x = ttnn.layer_norm(x, weight=self.ln_final_weights, bias=self.ln_final_bias)

        # TODO: Change to TTNN
        # text_projection = ttnn.transpose(self.text_projection, -2, -1)

        torch_tokens = ttnn.to_torch(tokens)
        text_projection = ttnn.to_torch(self.text_projection)
        torch_x = ttnn.to_torch(x)

        torch_x = torch_x[torch.arange(torch_x.shape[0]), torch_tokens.argmax(dim=-1)] @ text_projection.t()

        return ttnn.from_torch(torch_x, device=get_device(), layout=ttnn.TILE_LAYOUT)

    def forward(self, image, tokens):
        text_features = self.encode_text(tokens)
        image_features = self.encode_image(image)

        # Normalize features
        norm_image_features = ttnn.operations.moreh.norm(image_features, p=2.0, dim=1, keepdim=True)
        norm_text_features = ttnn.operations.moreh.norm(text_features, p=2.0, dim=1, keepdim=True)

        image_features = ttnn.divide(image_features, norm_image_features)
        text_features = ttnn.divide(text_features, norm_text_features)

        # Cosine similarity as logits
        logit_scale = math.exp(self.logit_scale)

        text_features_t = ttnn.transpose(text_features, 0, 1)
        logits_per_image = logit_scale * image_features @ text_features_t
        logits_per_text = ttnn.transpose(logits_per_image, 0, 1)

        return logits_per_image, logits_per_text


While input images can have any dimension and color spaces, our CLIP models only handles 224x224 RGB images. 

We therefore pre-process the image to the dimensions and RGB colorspace of the model. 

In [ ]:
def preprocess_image(image, model_resolution):
    def _convert_image_to_rgb(image):
        return image.convert("RGB")

    # Pre-process image on host with torch
    transform_fn = Compose(
        [
            Resize(model_resolution, interpolation=InterpolationMode.BICUBIC),
            CenterCrop(model_resolution),
            _convert_image_to_rgb,
            ToTensor(),
            Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        ]
    )
    return transform_fn(image)

We use an utility function to download image an URL.

In [ ]:
def download_image(url):
    """
    Download an image from a URL and return it as a PIL Image object.
    
    Args:
        url (str): The URL of the image to download
        
    Returns:
        PIL.Image: The downloaded image
    """
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Convert the response content to a PIL Image
        image = Image.open(BytesIO(response.content))
        return image
    except requests.RequestException as e:
        raise Exception(f"Failed to download image from {url}: {e}")
    except Exception as e:
        raise Exception(f"Failed to process downloaded image: {e}")

Having defined each step of our CLIP model, we can now use it on a input image and text. 

The weights are automatically downloaded using `CLIPModel.from_pretrained()`. 

We also tokenize text inputs using the tokenizer model that matches our CLIP model. In this case, we download the `openai/clip-vit-base-patch32` tokenizer model using `CLIPTokenizer.from_pretrained()`.
As TT-NN does not handle tokenization, we use `CLIPTokenizer` from the `transformers` library to handle it for us. 
We then convert the output of `tokenizer()` to TT-NN tensors. 

In [ ]:
if __name__ == "__main__":
    open_ttnn()

    logging_file = open("logging.csv", "w")

    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    state_dict = convert_model_to_ttnn(model.state_dict())

    clip = CLIP(state_dict)

    # Download image from URL
    image_url = "https://media.githubusercontent.com/media/tenstorrent/tutorial-assets/nmaurice/clip-tutorial/media/clip_tutorial/CLIP.png"
    image = download_image(image_url)

    # Preprocess image
    image = preprocess_image(image, 224).unsqueeze(0).to("cpu")

    preferred_dtype = ttnn.bfloat16
    tt_image = to_ttnn(image, preferred_dtype)

    prompts = ["a diagram", "a dog", "a cat"]

    # Tokenize text
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
    tokenized_inputs = tokenizer(prompts, padding="max_length", max_length=clip.context_length, return_tensors="pt")
    tokens_pretrained_host = tokenized_inputs["input_ids"]
    tokens_pretrained = ttnn.from_torch(tokens_pretrained_host, device=get_device(), layout=ttnn.TILE_LAYOUT)

    logits_per_image, logits_per_text = clip.forward(tt_image, tokens_pretrained)
    probs = ttnn.softmax(logits_per_image, dim=-1)
    print(f"Label probs: {probs}")

    logging_file.close()

    close_ttnn()
